In [1]:
from dwave.system.samplers import DWaveSampler
from minorminer import find_embedding
from dwave.system.composites import FixedEmbeddingComposite
import numpy as np

In [2]:
solver = "DW_2000Q_6"
sampler = DWaveSampler(solver=solver)

qubo = dok_matrix([[-0.25, 1], [0, -0.25]])


offset0 = -0.1
offset1 = 0
tries = 100

In [3]:
qpu_graph = sampler.edgelist
anneal_offset = np.zeros(2048)
for _ in range(tries):
    embedding = find_embedding(qubo_dict, qpu_graph)
    embedding_idx = [idx for embed_list in embedding.values() for idx in embed_list]
    embed = FixedEmbeddingComposite(sampler, embedding)
    anneal_offset_ranges = np.array(
        embed.properties["child_properties"]["anneal_offset_ranges"]
    )
    offset_range = anneal_offset_ranges[embedding_idx]
    range0 = offset_range[0][0] <= offset0 <= offset_range[0][1]
    range1 = offset_range[1][0] <= offset1 <= offset_range[1][1]
    if range0 and range1:
        anneal_offset[embedding_idx[0]] = offset0
        anneal_offset[embedding_idx[1]] = offset1
        break
print(anneal_offset_ranges[embedding_idx])
print(embedding)
print(qubo_dict)

NameError: name 'qubo_dict' is not defined

In [ ]:
dwave_config = {
    "annealing_time": 1,  # integer microseconds [1, 2000]
    "answer_mode": "histogram",  # histogram or raw
    "auto_scale": False,
    "num_reads": 1000,  # raw will dump out all results
    "num_spin_reversal_transforms": 0,
    "readout_thermalization": 0,  # default 0
    "programming_thermalization": 1000,  # default 1000
    "chain_strength": 1,
    "anneal_offsets": list(anneal_offset),
}

In [ ]:
result = embed.sample_qubo(qubo_dict, **dwave_config)
raw = result.to_pandas_dataframe()

In [ ]:
raw